## This notebook we will be cleaning the data of any NaN values, and getting it ready for machine learning applications.

In [1]:
#Running data ingest python notebook

%run /Users/cristianswift/Desktop/armbrust-lab/Seaflow-Machine-Learning/python/01_data-ingest-cmap.ipynb

### Need to ajust columns that contain numbers to floats

In [2]:
def ChangeObjectTypes(df):
    for column in df:
        if column == 'time':
            #changing to datetime
            df[column] = pd.to_datetime(df[column], errors='coerce')
            
        elif column == 'population' or column == 'cruisename':
            #changing to string
            df[column] = df[column].astype(str)
            
        else:
            #changing to numeric type
            df[column] = pd.to_numeric(df[column], errors='coerce')
    return df
ChangeObjectTypes(tn413)

,population,time,lat,lon,abundance_cells_per_microliter,diameter_micrometer,depth,NO3NO2,PO4,Fe,SiO4,Satellite_CHL,temp,salin
0,picoeuk,2023-02-25 05:40:00,21.306700,-157.036600,14.303231,1.557860,0.0,0.000562,0.212505,0.000433,2.092356,0.072018,24.03829,34.93311
1,picoeuk,2023-02-25 05:50:00,21.314200,-157.032100,20.716981,1.576000,0.0,0.000562,0.212505,0.000433,2.092356,0.072018,24.01350,34.92320
2,prochloro,2023-02-25 05:50:00,21.314100,-157.031633,64.269941,0.671474,0.0,0.000562,0.212505,0.000433,2.092356,0.072018,24.01350,34.92320
3,picoeuk,2023-02-25 06:00:00,21.312300,-157.016800,22.590023,1.606258,0.0,0.000562,0.212505,0.000433,2.092356,0.072018,23.98739,34.93976
4,prochloro,2023-02-25 06:00:00,21.312500,-157.019100,71.501965,0.678254,0.0,0.000562,0.212505,0.000433,2.092356,0.072018,23.98739,34.93976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4791,prochloro,2023-03-08 19:30:00,-8.781667,-180.452400,44.943551,0.549335,0.0,0.001317,0.313235,0.000020,2.038989,0.052806,29.71862,35.29352
4792,synecho,2023-03-08 19:30:00,-8.790600,-180.456000,2.894701,0.868585,0.0,0.001317,0.313235,0.000020,2.038989,0.052806,29.71862,35.29352
4793,picoeuk,2023-03-08 19:40:00,-8.799500,-180.459600,8.173275,1.539285,0.0,0.001317,0.313235,0.000020,2.038989,0.052806,29.72274,35.28927
4794,prochloro,2023-03-08 19:40:00,-8.799500,-180.459600,42.654278,0.546215,0.0,0.001317,0.313235,0.000020,2.038989,0.052806,29.72274,35.28927


### Now all of the objects are in the right format

In [3]:
print(pd.DataFrame(tn413))
print('\n')
print(tn413.dtypes)
print('\n')

     population                time        lat         lon  \
0       picoeuk 2023-02-25 05:40:00  21.306700 -157.036600   
1       picoeuk 2023-02-25 05:50:00  21.314200 -157.032100   
2     prochloro 2023-02-25 05:50:00  21.314100 -157.031633   
3       picoeuk 2023-02-25 06:00:00  21.312300 -157.016800   
4     prochloro 2023-02-25 06:00:00  21.312500 -157.019100   
...         ...                 ...        ...         ...   
4791  prochloro 2023-03-08 19:30:00  -8.781667 -180.452400   
4792    synecho 2023-03-08 19:30:00  -8.790600 -180.456000   
4793    picoeuk 2023-03-08 19:40:00  -8.799500 -180.459600   
4794  prochloro 2023-03-08 19:40:00  -8.799500 -180.459600   
4795    synecho 2023-03-08 19:40:00  -8.799500 -180.459600   

      abundance_cells_per_microliter  diameter_micrometer  depth    NO3NO2  \
0                          14.303231             1.557860    0.0  0.000562   
1                          20.716981             1.576000    0.0  0.000562   
2                    

## Calculating Biomaass

### Seaflow uses 
1. Cell abundance
2. Cell Diameter

To Calulate Biomass we first need to take the diameter of the plankton and find the volume using:
d = diameter
r = radius
QC = Carbon Quotas


1. $\frac{Diameter}{2}\ = r$

2. $\frac{4}{3}\pi r^3 = biovolume$

3. *Qc = 0.261 * Biovolume <sup>0.86</sup>*

4. $biomass = Qc * abundance$

In [4]:
#first calulating volumn of a sphere given plankton diamter
import math
plankton_volume = (4/3) * math.pi * (tn413['diameter_micrometer']/2)**3
#calulating carbon quota
carbon_quota = 0.261 * plankton_volume**0.86
# multipling the result by abundance to get total biomass per measurement (multipling by 1000 to get pgC per L)
tn413['biomass'] = tn413['abundance_cells_per_microliter'] * carbon_quota

#dropping the uneeeded depth column
tn413.drop('depth', axis=1, inplace=True)

In [5]:
tn413

,population,time,lat,lon,abundance_cells_per_microliter,diameter_micrometer,NO3NO2,PO4,Fe,SiO4,Satellite_CHL,temp,salin,biomass
0,picoeuk,2023-02-25 05:40:00,21.306700,-157.036600,14.303231,1.557860,0.000562,0.212505,0.000433,2.092356,0.072018,24.03829,34.93311,6.716405
1,picoeuk,2023-02-25 05:50:00,21.314200,-157.032100,20.716981,1.576000,0.000562,0.212505,0.000433,2.092356,0.072018,24.01350,34.92320,10.023072
2,prochloro,2023-02-25 05:50:00,21.314100,-157.031633,64.269941,0.671474,0.000562,0.212505,0.000433,2.092356,0.072018,24.01350,34.92320,3.441297
3,picoeuk,2023-02-25 06:00:00,21.312300,-157.016800,22.590023,1.606258,0.000562,0.212505,0.000433,2.092356,0.072018,23.98739,34.93976,11.478879
4,prochloro,2023-02-25 06:00:00,21.312500,-157.019100,71.501965,0.678254,0.000562,0.212505,0.000433,2.092356,0.072018,23.98739,34.93976,3.929065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4791,prochloro,2023-03-08 19:30:00,-8.781667,-180.452400,44.943551,0.549335,0.001317,0.313235,0.000020,2.038989,0.052806,29.71862,35.29352,1.433595
4792,synecho,2023-03-08 19:30:00,-8.790600,-180.456000,2.894701,0.868585,0.001317,0.313235,0.000020,2.038989,0.052806,29.71862,35.29352,0.301105
4793,picoeuk,2023-03-08 19:40:00,-8.799500,-180.459600,8.173275,1.539285,0.001317,0.313235,0.000020,2.038989,0.052806,29.72274,35.28927,3.720992
4794,prochloro,2023-03-08 19:40:00,-8.799500,-180.459600,42.654278,0.546215,0.001317,0.313235,0.000020,2.038989,0.052806,29.72274,35.28927,1.340725
